In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

In [0]:
# from google.colab import files

# uploaded = files.upload()

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [0]:
str_to_float_var = LabelEncoder()
str_to_float_var.fit(Y)
encoded_Y = str_to_float_var.transform(Y)

In [0]:
# create model, write code below
def create_baseline():
    network = Sequential()
    network.add(Dense(60, activation='relu', input_dim=60))
    network.add(Dense(1, activation='sigmoid'))
   
    # Compile model, write code below
    network.compile(optimizer='Adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
    return network

In [30]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)

kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 79.82% (0.77%)


In [32]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 84.60% (1.11%)


In [33]:
# smaller model
def create_smaller():
    # create model, write code below
    network = Sequential()
    network.add(Dense(30, activation='relu', input_dim=60))
    network.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
   
    # Compile model, write code below
    network.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return network    
    
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 84.60% (1.11%)


In [34]:
# larger model
def create_larger():
    # create model, write code below
    network = Sequential()
    network.add(Dense(60, activation='relu', input_dim=60))
    network.add(Dense(30, activation='sigmoid'))
    network.add(Dense(1, activation='sigmoid'))
   
    # Compile model, write code below
    network.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return network    

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 83.17% (0.64%)


In [36]:
#This is perform on original model, not on smaller or larger models.
network_return = create_baseline()
network_return.fit(X, encoded_Y, epochs=50, batch_size=8, verbose=0)